In [ ]:
# Import necessary libraries
import os
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import mmread

# Setup all paths
out_path = "../out/AllonKleinLab/Experiment1"
if not os.path.exists(out_path):
    os.makedirs(out_path)

# Setup testing phase
TEST = False
TEST_genes = 300
TEST_samples = 500

# Archetypes
k = 15

# Load Dataset

In [ ]:
# Binary matrix indicating clonal membership of each cell
# The rows of this file represent cells and correspond to the rows of _counts_matrix_in_vitro_ (above).
# The columns represent clones. Not every cell belongs to a clone. 
# clone_matrix = mmread("data/AllonKleinLab/Experiment1/stateFate_inVitro_clone_matrix.mtx")
# cell metadata : cell type annotation
# cell_metadata = pd.read_csv("data/AllonKleinLab/Experiment1/stateFate_inVitro_metadata.txt", sep="\t")
# gene names
# gene_names = pd.read_csv("data/AllonKleinLab/Experiment1/stateFate_inVitro_gene_names.txt", header=None)
# List of cells belonging to the neutrophil/monocyte trajectory that were used in benchmark analysis
# neutrophil_monocyte_trajectory = pd.read_csv("data/AllonKleinLab/Experiment1/stateFate_inVitro_neutrophil_monocyte_trajectory.txt", sep="\t")
# pseudotime for neutrophil trajectory cells
# neutrophil_pseudotime = pd.read_csv("data/AllonKleinLab/Experiment1/stateFate_inVitro_neutrophil_pseudotime.txt", sep="\t")

# Number of transcripts for each gene in each cell
se = mmread("../data/AllonKleinLab/Experiment1/stateFate_inVitro_normed_counts.mtx").T

if TEST:
    tgenes = min(TEST_genes, se.shape[0])
    tsamples = min(TEST_samples, se.shape[1])
    se = se[:tgenes, :tsamples]

# Convert the matrix to AnnData object
adata = sc.AnnData(se)

# Standard workflow for pre-processing and clustering
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)


# Visualize data

In [ ]:
# PCA plot
imgname = os.path.join(out_path, "PCA.png")
print(f"Saving Image --- {imgname}")
sc.pl.pca(adata, save=imgname.split('/')[-1])

In [ ]:
# UMAP plot
imgname = os.path.join(out_path, "UMAP.png")
print(f"Saving Image --- {imgname}")
sc.pl.umap(adata, save=imgname.split('/')[-1])

In [ ]:
# Elbow plot
imgname = os.path.join(out_path, "elbow.pdf")
print(f"Saving Image --- {imgname}")
sc.pl.pca_variance_ratio(adata, log=True, save=imgname.split('/')[-1])